# Advanced LangChain Tutorial: Structured Outputs & Agents

This notebook covers advanced LangChain features:
- **Structured Outputs** with TypedDict and Pydantic models
- **Nested Data Structures** for complex responses
- **Agent Workflows** with tools and middleware
- **Memory Management** with checkpointers
- **Human-in-the-Loop** review workflows
- **Email Generation** with SMTP integration
- **Token Counting** and optimization
- **Production-Ready Patterns** for real-world applications

## Setup and Installation

Install required packages:
```bash
pip install langchain-anthropic langchain-ollama python-dotenv langchain langchain-core pydantic typing-extensions langgraph
```

In [2]:
# Import all required libraries
from dotenv import load_dotenv
import os
from typing_extensions import TypedDict, Annotated
from typing import List, Literal
from pydantic import BaseModel, Field
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import MemorySaver
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load environment variables
load_dotenv('.env')

True

## Model Configuration

Set up the model with support for both Anthropic and Ollama.

In [7]:
# Choose your model provider
USE_OLLAMA = True  # Set to True for Ollama, False for Anthropic

if USE_OLLAMA:
    model = ChatOllama(
        model="llama3.2",
        base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
        temperature=0.7,
    )
else:
    model = ChatAnthropic(
        model="claude-3-5-sonnet-20241022",
        temperature=0.7,
        max_tokens=4000,
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )

print(f"Model initialized: {'Ollama' if USE_OLLAMA else 'Anthropic Claude'}")

Model initialized: Ollama


# Part 1: Structured Outputs with TypedDict

## Overview
Structured outputs ensure the LLM returns data in a specific format using TypedDict or Pydantic models. This is essential for:
- Type safety
- Validation
- Integration with downstream systems
- Consistent data structures

## Example 1: Movie Details with TypedDict

Define structured response formats using TypedDict for movie information.

In [8]:
# Define Actor structure using TypedDict
class Actor(TypedDict):
    """Actor information in a movie"""
    name: Annotated[str, "The name of the actor"]
    age: Annotated[int, "The age of the actor"]
    awards: Annotated[int, "Number of awards won by the actor"]

# Define MovieDetails structure
class MovieDetails(TypedDict):
    """Complete movie information"""
    title: Annotated[str, "The title of the movie"]
    director: Annotated[str, "The director of the movie"]
    year: Annotated[int, "The release year of the movie"]
    genre: Annotated[str, "The genre of the movie"]
    rating: Annotated[float, "The IMDb rating of the movie"]
    actors: Annotated[List[Actor], "List of main actors in the movie"]

# Bind the structured output to the model
model_with_nested_typed_dict_output = model.with_structured_output(MovieDetails)

# Invoke the model
response = model_with_nested_typed_dict_output.invoke(
    "Provide detailed information about the web series 'Better Call Saul', including main actors."
)

print(response)

{'title': 'Better Call Saul', 'director': 'Vince Gilligan and Peter Gould', 'year': 2015, 'genre': 'Crime, Drama', 'rating': 9.1, 'actors': [{'name': 'Bob Odenkirk', 'age': 57}, {'name': 'Jonathan Banks', 'age': 73}]}


### Understanding TypedDict vs Pydantic

**TypedDict:**
- Lightweight type hints for dictionaries
- No runtime validation
- Part of Python's typing module
- Best for simple structures

**Pydantic:**
- Full data validation at runtime
- Rich feature set (default values, validators, etc.)
- Better error messages
- Best for complex, validated data structures

# Part 2: Structured Outputs with Pydantic

Pydantic provides more robust data validation and better developer experience.

## Example 2: Movie Details with Pydantic BaseModel

In [ ]:
# Define Movie structure using Pydantic
class Movie(BaseModel):
    """Movie model with validation"""
    title: str = Field(..., description="The title of the movie")
    director: str = Field(..., description="The director of the movie")
    year: int = Field(..., description="The release year of the movie")
    genre: str = Field(..., description="The genre of the movie")
    rating: float = Field(..., description="The IMDb rating of the movie")

# Define Actor structure using Pydantic
class Actor(BaseModel):
    """Actor model with validation"""
    name: str = Field(..., description="The name of the actor")
    age: int = Field(..., description="The age of the actor")
    awards: int = Field(..., description="Number of awards won by the actor")

# Define MovieDetails with nested Actor list
class MovieDetails(BaseModel):
    """Complete movie details with actors"""
    title: str = Field(..., description="The title of the movie")
    director: str = Field(..., description="The director of the movie")
    year: int = Field(..., description="The release year of the movie")
    genre: str = Field(..., description="The genre of the movie")
    rating: float = Field(..., description="The IMDb rating of the movie")
    actors: List[Actor] = Field(..., description="List of main actors in the movie")

# Bind Pydantic model to the LLM
model_with_nested_output = model.with_structured_output(MovieDetails)

# Get structured response
response = model_with_nested_output.invoke(
    "Provide detailed information about the web series 'Better Call Saul', including main actors."
)

print(response)

## Example 3: Contact Information Extraction

In [9]:
# Define ContactInfo Pydantic model
class ContactInfo(BaseModel):
    """Contact information structure"""
    name: str = Field(..., description="Full name of the contact")
    email: str = Field(..., description="Email address of the contact")
    phone: str = Field(..., description="Phone number of the contact")

# Create an agent that collects contact information
agent = create_agent(
    model=model,
    system_prompt="You are a helpful assistant that collects contact information.",
    response_format=ContactInfo
)

# Invoke the agent
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Collect contact info for John Doe, email john.doe@example.com, phone 123-456-7890"
            }
        ]
    }
)

print("\n=== Agent Result ===")
print(result)

# Access structured response
print("\n=== Extracted Contact Info ===")
print(result['structured_response'])


=== Agent Result ===
{'messages': [HumanMessage(content='Collect contact info for John Doe, email john.doe@example.com, phone 123-456-7890', additional_kwargs={}, response_metadata={}, id='7e74b608-69bf-4834-8f36-9ae4c4a81793'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2026-02-01T22:57:18.468668Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3371601208, 'load_duration': 23901583, 'prompt_eval_count': 217, 'prompt_eval_duration': 1948000000, 'eval_count': 39, 'eval_duration': 1142000000, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--019c1b6c-b54f-7122-8112-b4b4a32d9510-0', tool_calls=[{'name': 'ContactInfo', 'args': {'email': 'john.doe@example.com', 'name': 'John Doe', 'phone': '123-456-7890'}, 'id': 'fbc57da4-94cc-469d-a947-f38b9c9f818a', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 217, 'output_tokens': 39, 'total_tokens': 256}), ToolMessage(con

# Part 3: Agent Middleware - Summarization

## What is Middleware?
Middleware intercepts and processes messages between the user and the agent. Common uses:
- **Summarization**: Condense long conversation histories
- **Translation**: Convert between languages
- **Filtering**: Remove sensitive information
- **Logging**: Track conversations

## SummarizationMiddleware
Keeps conversation context manageable by summarizing old messages.

In [ ]:
# Check SummarizationMiddleware signature
print(SummarizationMiddleware.__init__.__doc__)

# Part 4: Advanced Agent with Tools and Middleware

Build a production-ready financial analysis agent with:
- Custom tools for ETF search
- Structured output (Pydantic models)
- Memory management
- Conversation summarization
- Token counting

## Define Pydantic Models for Financial Analysis

In [10]:
# ETF Analysis model
class ETFAnalysis(BaseModel):
    """Structured ETF analysis with required financial metrics."""
    etf_name: str = Field(..., description="ETF ticker (e.g., VOO)")
    one_year_return: float = Field(..., ge=0, description="1-year return percentage")
    five_year_return: float = Field(..., ge=0, description="5-year annualized return percentage")
    revenue_2023: float = Field(..., ge=0, description="Revenue for 2023 ($M)")
    revenue_2022: float = Field(..., ge=0, description="Revenue for 2022 ($M)")
    revenue_2021: float = Field(..., ge=0, description="Revenue for 2021 ($M)")
    total_assets: float = Field(..., ge=0, description="Total assets under management ($B)")

# Top 3 ETFs per company
class TopETFsAnalysis(BaseModel):
    """Top 3 ETFs per company."""
    company: str = Field(..., description="Company name")
    etf_1: ETFAnalysis = Field(..., description="Rank 1 ETF")
    etf_2: ETFAnalysis = Field(..., description="Rank 2 ETF")
    etf_3: ETFAnalysis = Field(..., description="Rank 3 ETF")

# Email content model
class EmailContent(BaseModel):
    """Email content for human review."""
    subject: str = Field(..., description="Email subject line")
    body: str = Field(..., description="Email body content")
    recipient: str = Field(..., description="Recipient email address")

## Define ETF Search Tool

In [11]:
@tool
def search_etfs(company: str) -> str:
    """Search top performing ETFs for the company. Returns ticker symbols."""
    # Simulated ETF database
    etfs = "VOO, VTI, SCHB, SPY, QQQ, VUG, VYM, FNCL, FTEC, XLK, XLF, SCHX, SCHD, IVV, IWF, RSP, ITOT"
    return f"Top ETFs for {company}: {etfs}"

## Token Counting for Production Use

Token counting is critical for:
- Cost management
- Context window optimization
- Performance monitoring

In [12]:
def make_token_counter(model):
    """Factory function to create token counter with model in closure."""
    
    def count_tokens(messages):
        """Token counter for middleware - only takes messages parameter."""
        try:
            # Try to use the model's built-in token counting method
            return model.get_num_tokens_from_messages(messages)
        except:
            # Fallback 1: Character-based estimation (accurate enough for most cases)
            total_chars = sum(len(str(getattr(m, 'content', ''))) for m in messages)
            return total_chars // 4 + 100  # ~4 chars per token + padding for headers/metadata
    
    return count_tokens

# Create the token counter for this model
token_counter = make_token_counter(model)

## Create Production Agent with All Features

In [13]:
# IMPORTANT: Structured outputs work differently for Anthropic vs Ollama
# - Anthropic: Native structured output support
# - Ollama: Returns text/code that needs parsing

if USE_OLLAMA:
    # For Ollama: Use simple agent without structured output
    # We'll parse the response manually
    agent = create_agent(
        model=model,
        tools=[search_etfs],
        system_prompt="""Financial analyst. You MUST respond with ONLY a JSON object in this exact format:
{
  "company": "CompanyName",
  "etf_1": {"etf_name": "VOO", "one_year_return": 15.5, "five_year_return": 75.2, "revenue_2023": 1000.0, "revenue_2022": 950.0, "revenue_2021": 900.0, "total_assets": 500.5},
  "etf_2": {"etf_name": "VTI", "one_year_return": 14.2, "five_year_return": 72.1, "revenue_2023": 800.0, "revenue_2022": 750.0, "revenue_2021": 700.0, "total_assets": 400.3},
  "etf_3": {"etf_name": "SPY", "one_year_return": 13.8, "five_year_return": 70.5, "revenue_2023": 600.0, "revenue_2022": 550.0, "revenue_2021": 500.0, "total_assets": 300.2}
}

Process:
1. Call search_etfs tool FIRST
2. Pick EXACTLY 3 different ETFs from results
3. Provide realistic metrics for each
4. Return ONLY the JSON, no explanation""",
        checkpointer=MemorySaver(),
    )
else:
    # For Anthropic: Use native structured output
    agent = create_agent(
        model=model,
        tools=[search_etfs],
        system_prompt="""Financial analyst. Process:
1. Call search_etfs FIRST
2. Pick EXACTLY top 3 ETFs from results
3. Realistic metrics: 1yr/5yr returns, revenue 2021-23, AUM
4. Return TopETFsAnalysis (etf_1, etf_2, etf_3 exactly)
Concise.""",
        checkpointer=MemorySaver(),
        response_format=TopETFsAnalysis,  # Structured output
        middleware=[
            SummarizationMiddleware(
                model=model,
                max_tokens_before_summary=900,
                messages_to_keep=10,
                token_counter=token_counter
            )
        ]
    )

# Configuration for threading
config = {"configurable": {"thread_id": "etf-final-v1"}}

## Helper Function: Parse Agent Response

Handle responses differently based on model provider.

In [14]:
import json
import re

def parse_agent_response(response, use_ollama=False):
    """
    Parse agent response for both Anthropic and Ollama.
    
    Args:
        response: Agent response dict
        use_ollama: Whether using Ollama (needs JSON parsing)
    
    Returns:
        TopETFsAnalysis object
    """
    if not use_ollama:
        # Anthropic: Direct structured response
        return response["structured_response"]
    
    # Ollama: Parse JSON from text response
    messages = response.get("messages", [])
    if not messages:
        raise ValueError("No messages in response")
    
    # Get the last AI message
    last_message = None
    for msg in reversed(messages):
        if hasattr(msg, 'content') and msg.content:
            last_message = msg.content
            break
    
    if not last_message:
        raise ValueError("No content in response")
    
    # Extract JSON from response (may be wrapped in code blocks or text)
    # Try to find JSON object
    json_match = re.search(r'\{[^{}]*"company"[^{}]*\{.*?\}.*?\{.*?\}.*?\{.*?\}.*?\}', last_message, re.DOTALL)
    
    if json_match:
        json_str = json_match.group(0)
        try:
            data = json.loads(json_str)
            # Convert to TopETFsAnalysis
            return TopETFsAnalysis(**data)
        except json.JSONDecodeError as e:
            print(f"JSON parse error: {e}")
            print(f"Attempted to parse: {json_str[:200]}...")
    
    # Fallback: Create mock data if parsing fails
    print("⚠️ Warning: Could not parse structured response, using mock data")
    return TopETFsAnalysis(
        company="Unknown",
        etf_1=ETFAnalysis(
            etf_name="VOO",
            one_year_return=15.5,
            five_year_return=75.0,
            revenue_2023=1000.0,
            revenue_2022=950.0,
            revenue_2021=900.0,
            total_assets=500.0
        ),
        etf_2=ETFAnalysis(
            etf_name="VTI",
            one_year_return=14.0,
            five_year_return=72.0,
            revenue_2023=800.0,
            revenue_2022=750.0,
            revenue_2021=700.0,
            total_assets=400.0
        ),
        etf_3=ETFAnalysis(
            etf_name="SPY",
            one_year_return=13.0,
            five_year_return=70.0,
            revenue_2023=600.0,
            revenue_2022=550.0,
            revenue_2021=500.0,
            total_assets=300.0
        )
    )

## Run Financial Analysis for Multiple Companies

In [15]:
# List of companies to analyze
companies = ["Vanguard", "Fidelity", "State Street", "Charles Schwab", "Invesco", "Blackrock"]

print("🔍 TOP 3 ETFs ANALYSIS (6 Companies = 18 ETFs)")
print("=" * 90)

all_results = []

for company in companies:
    print(f"\n📊 Analyzing {company}...")
    
    # Invoke agent for this company
    response = agent.invoke(
        {
            "messages": [
                HumanMessage(
                    content=f"Analyze top ETFs from {company}: provide ETF name, 1yr/5yr returns, revenue past 3 years, total assets"
                )
            ]
        },
        config=config
    )
    
    # Parse response (handles both Anthropic and Ollama)
    try:
        analysis = parse_agent_response(response, use_ollama=USE_OLLAMA)
    except Exception as e:
        print(f"⚠️ Error parsing response for {company}: {e}")
        print(f"Response keys: {response.keys()}")
        continue
    
    tokens = token_counter(response.get("messages", []))
    
    # Display results
    print(f"  📈 {analysis.etf_1.etf_name:<6} | "
          f"1yr:{analysis.etf_1.one_year_return:5.1f}% "
          f"5yr:{analysis.etf_1.five_year_return:5.1f}% "
          f"AUM:${analysis.etf_1.total_assets:7.1f}B")
    
    print(f"  📈 {analysis.etf_2.etf_name:<6} | "
          f"1yr:{analysis.etf_2.one_year_return:5.1f}% "
          f"5yr:{analysis.etf_2.five_year_return:5.1f}% "
          f"AUM:${analysis.etf_2.total_assets:7.1f}B")
    
    print(f"  📈 {analysis.etf_3.etf_name:<6} | "
          f"1yr:{analysis.etf_3.one_year_return:5.1f}% "
          f"5yr:{analysis.etf_3.five_year_return:5.1f}% "
          f"AUM:${analysis.etf_3.total_assets:7.1f}B")
    
    print(f"  📊 Tokens: ~{tokens} (msgs: {len(response['messages'])})")
    
    # Store results
    all_results.append({
        'company': analysis.company,
        'etf1': analysis.etf_1.model_dump(),  # Fixed: Pydantic V2
        'etf2': analysis.etf_2.model_dump(),  # Fixed: Pydantic V2
        'etf3': analysis.etf_3.model_dump(),  # Fixed: Pydantic V2
        'tokens': tokens
    })

🔍 TOP 3 ETFs ANALYSIS (6 Companies = 18 ETFs)

📊 Analyzing Vanguard...


/Users/aniketanvekar/Documents/LangChain/LangChain/.venv/lib/python3.11/site-packages/langchain_core/language_models/base.py:328: UserWarning: Using fallback GPT-2 tokenizer for token counting. Token counts may be inaccurate for non-GPT-2 models. For accurate counts, use a model-specific method if available.
  return len(self.get_token_ids(text))


  📈 VOO    | 1yr: 15.5% 5yr: 75.2% AUM:$  500.5B
  📈 VTI    | 1yr: 14.2% 5yr: 72.1% AUM:$  400.3B
  📈 SCHB   | 1yr: 12.8% 5yr: 68.4% AUM:$  200.5B
  📊 Tokens: ~291 (msgs: 4)

📊 Analyzing Fidelity...
  📈 VOO    | 1yr: 15.5% 5yr: 75.2% AUM:$  500.5B
  📈 VTI    | 1yr: 14.2% 5yr: 72.1% AUM:$  400.3B
  📈 FTEC   | 1yr: 10.5% 5yr: 63.4% AUM:$  150.2B
  📊 Tokens: ~483 (msgs: 8)

📊 Analyzing State Street...
  📈 VOO    | 1yr: 15.5% 5yr: 75.2% AUM:$  500.5B
  📈 VTI    | 1yr: 14.2% 5yr: 72.1% AUM:$  400.3B
  📈 SPY    | 1yr: 13.8% 5yr: 70.5% AUM:$  300.2B
  📊 Tokens: ~678 (msgs: 12)

📊 Analyzing Charles Schwab...
  📈 VOO    | 1yr: 15.5% 5yr: 75.2% AUM:$  500.5B
  📈 VTI    | 1yr: 14.2% 5yr: 72.1% AUM:$  400.3B
  📈 SCHB   | 1yr: 12.8% 5yr: 68.4% AUM:$  200.5B
  📊 Tokens: ~874 (msgs: 16)

📊 Analyzing Invesco...
  📈 VOO    | 1yr: 15.5% 5yr: 75.2% AUM:$  500.5B
  📈 VTI    | 1yr: 14.2% 5yr: 72.1% AUM:$  400.3B
  📈 VYM    | 1yr: 12.8% 5yr: 68.4% AUM:$  250.2B
  📊 Tokens: ~1065 (msgs: 20)

📊 Analyzing Blac

## Display Summary Table

In [ ]:
# Summary table
print("\n" + "=" * 100)
print("📊 SUMMARY | TOP 3 ETFs | 6 COMPANIES | 18 TOTAL")
print("=" * 100)

print(f"{'Company'.ljust(13)}|{'ETF1'.ljust(8)}|{'1yr'.ljust(6)}|\n"
      f"{'ETF2'.ljust(8)}|{'1yr'.ljust(6)}|\n"
      f"{'ETF3'.ljust(8)}|{'Tokens'}")
print("=" * 100)

for r in all_results:
    e1, e2, e3 = r['etf1'], r['etf2'], r['etf3']
    print(f"{r['company'].ljust(13)}|{e1['etf_name'].ljust(8)}|{e1['one_year_return']:5.1f}%|\n"
          f"{e2['etf_name'].ljust(8)}|{e2['one_year_return']:5.1f}%|\n"
          f"{e3['etf_name'].ljust(8)}|{r['tokens']:5d}")

print("=" * 100)

## Export Results to Files

In [ ]:
# Export to Pandas DataFrame and CSV
import pandas as pd

print("\n✅ PERFECT! Data ready in 'all_results'")
print("📊 Pandas: pd.DataFrame([{**r, 'all_etfs':[r['etf1'],r['etf2'],r['etf3']]} for r in all_results])")
print("💾 CSV: df.to_csv('top3_etfs_6companies.csv')")

# Part 5: Human-in-the-Loop Email Workflow

## Why Human-in-the-Loop?
For sensitive operations like sending emails, human review ensures:
- Content accuracy
- Tone appropriateness
- Compliance with policies
- Error prevention

This pattern is essential for production AI systems.

## Email Content Generation Function

In [ ]:
def generate_email_content(analysis: TopETFsAnalysis, recipient: str) -> EmailContent:
    """Generate email content from ETF analysis."""
    subject = f"Top 3 ETF Recommendations from {analysis.company}"
    
    body = f"""
Dear Investor,

Based on our AI-powered analysis, here are the top 3 ETF recommendations from {analysis.company}:

🔥 #{1} - {analysis.etf_1.etf_name}
   • 1-Year Return: {analysis.etf_1.one_year_return:.1f}%
   • 5-Year Return: {analysis.etf_1.five_year_return:.1f}%
   • Assets Under Management: ${analysis.etf_1.total_assets:.1f}B
   • Revenue (2023): ${analysis.etf_1.revenue_2023:.0f}M

🔥 #{2} - {analysis.etf_2.etf_name}
   • 1-Year Return: {analysis.etf_2.one_year_return:.1f}%
   • 5-Year Return: {analysis.etf_2.five_year_return:.1f}%
   • Assets Under Management: ${analysis.etf_2.total_assets:.1f}B
   • Revenue (2023): ${analysis.etf_2.revenue_2023:.0f}M

🔥 #{3} - {analysis.etf_3.etf_name}
   • 1-Year Return: {analysis.etf_3.one_year_return:.1f}%
   • 5-Year Return: {analysis.etf_3.five_year_return:.1f}%
   • Assets Under Management: ${analysis.etf_3.total_assets:.1f}B
   • Revenue (2023): ${analysis.etf_3.revenue_2023:.0f}M

---
This analysis was generated by our AI financial advisor.
Please consult with a financial professional before making investment decisions.

Best regards,
Investment Analysis Team
"""
    
    return EmailContent(subject=subject, body=body, recipient=recipient)

## Human Review Function

Interactive function that allows humans to:
- Approve email for sending
- Edit subject or body
- Reject email

In [ ]:
def human_review_email(email_content: EmailContent) -> tuple[bool, EmailContent]:
    """
    Display email to human reviewer and get approval.
    Returns: (approved: bool, modified_email: EmailContent)
    """
    print("\n" + "=" * 80)
    print("📧 EMAIL REVIEW - HUMAN APPROVAL REQUIRED")
    print("=" * 80)
    print(f"\n📧 TO: {email_content.recipient}")
    print(f"📧 SUBJECT: {email_content.subject}")
    print(f"\n📧 BODY:\n{email_content.body}")
    print("\n" + "=" * 80)
    
    # Get human approval
    while True:
        decision = input(
            "\n🤔 Review Decision:\n [A]pprove [E]dit [R]eject\nYour choice: "
        ).strip().upper()
        
        if decision == 'A':
            print("✅ Email APPROVED for sending!")
            return True, email_content
        
        elif decision == 'E':
            print("\n📝 Edit Mode:")
            new_subject = input(
                f"New Subject (press Enter to keep '{email_content.subject}'): "
            ).strip()
            new_body = input(
                f"New Body (press Enter to keep current): "
            ).strip()
            
            if new_subject:
                email_content.subject = new_subject
            if new_body:
                email_content.body = new_body
            
            print("📧 Email updated! Showing revised version...")
            return human_review_email(email_content)  # Recursive review
        
        elif decision == 'R':
            print("❌ Email REJECTED - will not be sent")
            return False, email_content
        
        else:
            print("⚠️ Invalid choice. Please enter A, E, or R.")

## SMTP Email Sending Function

**⚠️ IMPORTANT**: This is a placeholder. Configure with your actual SMTP settings before production use.

In [ ]:
def send_email(email_content: EmailContent) -> bool:
    """
    Send email via SMTP.
    ⚠️ Configure with your SMTP settings before production use!
    """
    try:
        # EXAMPLE SMTP configuration (UPDATE THESE!)
        SMTP_SERVER = "smtp.gmail.com"  # Change to your SMTP server
        SMTP_PORT = 587
        SMTP_USER = "your-email@example.com"  # Your email
        SMTP_PASSWORD = "your-app-password"  # Your app password
        
        # Create message
        msg = MIMEMultipart()
        msg['From'] = SMTP_USER
        msg['To'] = email_content.recipient
        msg['Subject'] = email_content.subject
        msg.attach(MIMEText(email_content.body, 'plain'))
        
        # Send email (COMMENTED OUT FOR SAFETY - ENABLE WHEN READY)
        # server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        # server.starttls()
        # server.login(SMTP_USER, SMTP_PASSWORD)
        # server.send_message(msg)
        # server.quit()
        
        print(f"📧 Email sent successfully to {email_content.recipient}")
        return True
    
    except Exception as e:
        print(f"❌ Failed to send email: {e}")
        return False

## Complete Email Workflow

Put it all together: Analysis → Email Generation → Human Review → Send

In [ ]:
# Main workflow with HITL
config = {"configurable": {"thread_id": "etf-final-v1"}}
companies = ["Vanguard", "Fidelity", "State Street", "Charles Schwab", "Invesco", "Blackrock"]
recipient_email = "investor@example.com"  # Change to actual recipient

print("🔍 TOP 3 ETFs ANALYSIS WITH EMAIL NOTIFICATION")
print("=" * 90)

all_results = []
emails_sent = 0
emails_rejected = 0

for company in companies:
    print(f"\n📊 Analyzing {company}...")
    
    # Step 1: Get ETF analysis from agent
    response = agent.invoke(
        {
            "messages": [
                HumanMessage(
                    content=f"Top 3 ETFs for {company}: ticker+1yr/5yr returns+revenue 2021-23+AUM"
                )
            ]
        },
        config=config
    )
    
    analysis = response["structured_response"]
    tokens = token_counter(response["messages"])
    
    # Display results
    print(f"  📈 {analysis.etf_1.etf_name:<6} | "
          f"1yr:{analysis.etf_1.one_year_return:5.1f}% "
          f"5yr:{analysis.etf_1.five_year_return:5.1f}% "
          f"AUM:${analysis.etf_1.total_assets:7.1f}B")
    print(f"  📈 {analysis.etf_2.etf_name:<6} | "
          f"1yr:{analysis.etf_2.one_year_return:5.1f}% "
          f"5yr:{analysis.etf_2.five_year_return:5.1f}% "
          f"AUM:${analysis.etf_2.total_assets:7.1f}B")
    print(f"  📈 {analysis.etf_3.etf_name:<6} | "
          f"1yr:{analysis.etf_3.one_year_return:5.1f}% "
          f"5yr:{analysis.etf_3.five_year_return:5.1f}% "
          f"AUM:${analysis.etf_3.total_assets:7.1f}B")
    print(f"  📊 Tokens: ~{tokens} (msgs: {len(response['messages'])})")
    
    all_results.append({
        'company': analysis.company,
        'etf1': analysis.etf_1.model_dump(),
        'etf2': analysis.etf_2.model_dump(),
        'etf3': analysis.etf_3.model_dump(),
        'tokens': tokens
    })
    
    # Step 2: Generate email content
    email_content = generate_email_content(analysis, recipient_email)
    
    # Step 3: HUMAN-IN-THE-LOOP REVIEW
    approved, reviewed_email = human_review_email(email_content)
    
    # Step 4: Send email if approved
    if approved:
        # Uncomment next line when SMTP is configured
        # send_email(reviewed_email)
        print("⚠️ SMTP not configured - email simulated")
        emails_sent += 1
    else:
        emails_rejected += 1

## Final Summary

In [ ]:
# Final summary
print("\n" + "=" * 100)
print("📊 FINAL SUMMARY")
print("=" * 100)
print(f"✅ Companies Analyzed: {len(companies)}")
print(f"📧 Emails Sent: {emails_sent}")
print(f"❌ Emails Rejected: {emails_rejected}")
print(f"📊 Total ETFs Recommended: {len(companies) * 3}")

# Summary table
print("\n" + "=" * 100)
print("📊 ETF SUMMARY TABLE")
print("=" * 100)
print(f"{'Company'.ljust(13)}|{'ETF1'.ljust(8)}|{'1yr'.ljust(6)}|{'ETF2'.ljust(8)}|{'1yr'.ljust(6)}|{'ETF3'.ljust(8)}|{'Tokens'}")
print("=" * 100)

for r in all_results:
    e1, e2, e3 = r['etf1'], r['etf2'], r['etf3']
    print(f"{r['company'].ljust(13)}|{e1['etf_name'].ljust(8)}|{e1['one_year_return']:5.1f}%|"
          f"{e2['etf_name'].ljust(8)}|{e2['one_year_return']:5.1f}%|"
          f"{e3['etf_name'].ljust(8)}|{r['tokens']:5d}")

print("=" * 100)

# Summary: Key Concepts Covered

## 1. Structured Outputs
- **TypedDict**: Lightweight type hints for simple structures
- **Pydantic**: Full validation and rich features for production
- **Nested Models**: Complex hierarchical data structures

## 2. Agents and Tools
- **Custom Tools**: Define functions LLMs can call
- **Tool Binding**: Attach tools to models
- **Structured Responses**: Ensure consistent output format

## 3. Middleware
- **SummarizationMiddleware**: Manage conversation context
- **Token Counting**: Monitor and optimize costs
- **Memory Management**: Preserve important context

## 4. Human-in-the-Loop
- **Review Workflows**: Human approval for critical operations
- **Interactive Editing**: Modify AI outputs before execution
- **Safety Gates**: Prevent errors in production

## 5. Production Patterns
- **Configuration Management**: Thread IDs for conversation tracking
- **Error Handling**: Graceful fallbacks
- **Data Export**: CSV, Pandas integration
- **Email Integration**: SMTP configuration

## Best Practices
1. Always use structured outputs for production systems
2. Implement token counting to manage costs
3. Add human review for sensitive operations
4. Use middleware for long conversations
5. Validate all external inputs with Pydantic
6. Keep system prompts clear and concise
7. Test thoroughly before production deployment